In [148]:
import torch
import os
import random
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torch import GradScaler
from torch import nn, optim
import tqdm
import torchvision.transforms.functional

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [153]:
class SpaceNetDataset(Dataset):
    def __init__(self, folder):
        self.files = {}
        self.combinations = []

        for root, _, files in os.walk(folder):
            for file in files:
                year, month = file.split('_')[2:4]
                map_id = file.split('_')[-2]
                if map_id not in self.files:
                    self.files[map_id] = []
                image = Image.open(os.path.join(root, file))
                image_np = torchvision.transforms.functional.pil_to_tensor(image)
                self.files[map_id].append((int(year) * 12 + int(month), image_np))

        for place in self.files.values():
            for a in place:
                for b in place:
                    if a[0] < b[0]:
                        self.combinations.append((a, b))

        random.shuffle(self.combinations)

        self.count = len(self.combinations)

    def __len__(self):
        return len(self.combinations)

    def __getitem__(self, idx):
        (a_date, a_image), (b_date, b_image) = self.combinations[idx]

        rotations = torch.randint(0, 4, (1,))

        a_image = torchvision.transforms.functional.rotate(a_image, int(90 * rotations))
        b_image = torchvision.transforms.functional.rotate(b_image, int(90 * rotations))

        return (a_image, b_image, b_date - a_date)

dataset = SpaceNetDataset("./Advanced-Topics-in-Neural-Networks-Template-2024/Lab04/Dataset/")

In [154]:
class SplitDataset(SpaceNetDataset):
    def __init__(self, dataset: SpaceNetDataset, split_start, split_end):
        self.combinations = dataset.combinations[int(len(dataset.combinations) * split_start):int(len(dataset.combinations) * split_end)]

In [155]:
train_dataset = SplitDataset(dataset, 0, 0.7)
validate_dataset = SplitDataset(dataset, 0.7, 0.85)
test_dataset = SplitDataset(dataset, 0.85, 1)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
validate_dataloader = DataLoader(validate_dataset, batch_size=32, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=4)

In [162]:
for i in train_dataloader:
    print(i[2])
    break

tensor([13, 16,  3,  6,  7,  8, 18,  5,  1,  6,  9,  1, 23, 15,  1, 15, 14,  1,
        14,  9,  1,  1, 10,  1,  7,  9,  2, 11,  7, 12, 18,  5])


In [103]:
class SimpleModel(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, 3, padding=1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.layers(x)

In [105]:
pin_memory = True
enable_half = False
scaler = GradScaler(device, enabled=enable_half)

In [106]:
model = SimpleModel().to(device)
model = torch.jit.script(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0, fused=True)

In [107]:
def train():
    model.train()

    for a_image, b_image, time_skip in train_dataloader:
        a, b = a_image.to(device, non_blocking=True), b_image.to(device, non_blocking=True)
        with torch.autocast(device.type, enabled=enable_half):
            outputs = model(a)
            loss = criterion(outputs, b)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        print(loss)

train()

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tedy/Git/FII-AdvRN/venv/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/tedy/Git/FII-AdvRN/venv/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/tedy/Git/FII-AdvRN/venv/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tedy/Git/FII-AdvRN/venv/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py", line 174, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed] 

In [ ]:
@torch.inference_mode()
def val():
    model.eval()
    correct = 0
    total = 0

    for inputs, targets in validate_dataloader:
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        with torch.autocast(device.type, enabled=enable_half):
            outputs = model(inputs)

        predicted = outputs.argmax(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return 100.0 * correct / total

In [ ]:
@torch.inference_mode()
def inference():
    model.eval()

    labels = []

    correct = 0
    total = 0

    for inputs, targets in test_dataloader:
        inputs = inputs.to(device, non_blocking=True)
        with torch.autocast(device.type, enabled=enable_half):
            outputs = []
            outputs.append(model(inputs))
        stacked_outputs = torch.stack(outputs)
        predicted = torch.max(stacked_outputs, dim=0)[0].argmax(dim=1)

        correct += predicted.to("cpu").eq(targets).sum().item()
        total += predicted.size()[0]
        
        labels.extend(predicted.tolist())

    print(correct / total * 100.0)
    
    return labels

In [ ]:
best = 0.0
epochs = list(range(500))
with tqdm(epochs) as tbar:
    for epoch in tbar:
        train_acc = train()
        val_acc = val()
        if val_acc > best:
            best = val_acc
        tbar.set_description(f"Train: {train_acc:.2f}, Val: {val_acc:.2f}, Best: {best:.2f}")
        print(f"Train: {train_acc:.2f}, Val: {val_acc:.2f}, Best: {best:.2f}")